# Eodag: define a search area
____________

We present `eodag`'s features for **searching products that intersect an area of interest**.

To be able to follow this tutorial you will need to:
* Install the additional packages [ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/installation.html) and [ipywidgets](https://ipywidgets.readthedocs.io/en/stable/user_install.html)
* Download the auxiliary files [here](https://github.com/CS-SI/eodag/tree/master/examples/auxdata/) which contain an archived shapefile.


## Configuration

Let's start by setting your personal credentials to access [PEPS service](https://peps.cnes.fr) by filling your username and password below:

In [1]:
import os
os.environ["EODAG__PEPS__AUTH__CREDENTIALS__USERNAME"] = "PLEASE_CHANGE_ME"
os.environ["EODAG__PEPS__AUTH__CREDENTIALS__PASSWORD"] = "PLEASE_CHANGE_ME"

Let's check that `ipyleaflet` and `ipywidgets` are available:

In [2]:
import ipyleaflet
import ipywidgets

Then we create a workspace directory where all our files and configuration will live:

In [3]:
workspace = 'eodag_workspace_search_area'
if not os.path.isdir(workspace):
    os.mkdir(workspace)

You should have downloaded the **auxdata** folder and saved it next to this tutorial's file.

In [4]:
sentinel2_grid_zip = os.path.join('auxdata', 'sentinel2_tiling_grid_centroids.zip')
if not os.path.isfile(sentinel2_grid_zip):
    raise FileNotFoundError("Auxdata not found, please check your configuration.")

## Search

The first step before performing a search is to initialize the session by creating an `EODataAccessGateway` instance with the previous configuration file.

In [5]:
# To have some basic feedback on what eodag is doing, we configure logging to output minimum information
from eodag.utils.logging import setup_logging
setup_logging(verbose=1)

from eodag.api.core import EODataAccessGateway

dag = EODataAccessGateway()

2021-01-20 09:59:36,625-15s eodag.config                     [INFO    ] Loading user configuration from: /home/maxime/.config/eodag/eodag.yml
2021-01-20 09:59:36,919-15s eodag.core                       [INFO    ] Locations configuration loaded from /home/maxime/.config/eodag/locations.yml


Throughout of the tutorial we will search for *S2_MSI_L1C* products in June 2018.

In [6]:
product_type = 'S2_MSI_L1C'
start_date = '2018-06-01'
end_date = '2018-06-15'

### Geometry search

The first way to define an area of interest if by passing a geometry to the `search` method via its `geom` argument. This argument accepts the following different inputs:

* a `Shapely` geometry object
* a *Well-Known Text* (WKT) string
* a bounding box as a dictionary with keys: `"lonmin"`, `"latmin"`, `"lonmax"`, `"latmax"`
* a bounding box as a list with elements provided in the order [lonmin, latmin, lonmax, latmax]

The coordinates must be provided in the **WGS84** projection (EPSG: 4326).

#### Shapely geometry object and WKT string

We define an irregular polygon in the South-West of France and display it on an interactive map.

In [7]:
from ipyleaflet import Map, Polygon

polygon = Polygon(locations=[(45, 0), (43, 1), (43, 2), (46, 3)], style=dict(color="green"))

m = Map(center=(44, 0), zoom=6)
m.add_layer(polygon)
m

Map(center=[44, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

This polygon is converted into a `Shapely` polygon object.

In [8]:
from shapely.geometry import Polygon
aio_shapely_polyg = Polygon(polygon.locations)

# We need to flip the X-Y coordinates of the Ipyleaflet polygin to be compatible with Shapely
from shapely.ops import transform
aio_shapely_polyg = transform(lambda x, y: (y, x), aio_shapely_polyg)

We search for products that intersect with this polygon.

In [9]:
products, estimated_total_nbr_of_results = dag.search(
    productType=product_type,
    start=start_date,
    end=end_date,
    geom=aio_shapely_polyg
)

2021-01-20 09:59:37,222-15s eodag.core                       [INFO    ] Searching product type 'S2_MSI_L1C' on provider: peps
2021-01-20 09:59:37,223-15s eodag.plugins.search.qssearch    [INFO    ] Sending count request: https://peps.cnes.fr/resto/api/collections/S2ST/search.json?startDate=2018-06-01&completionDate=2018-06-15&geometry=POLYGON ((0.0000 45.0000, 1.0000 43.0000, 2.0000 43.0000, 3.0000 46.0000, 0.0000 45.0000))&productType=S2MSI1C&maxRecords=1&page=1
2021-01-20 09:59:39,528-15s eodag.plugins.search.qssearch    [INFO    ] Sending search request: https://peps.cnes.fr/resto/api/collections/S2ST/search.json?startDate=2018-06-01&completionDate=2018-06-15&geometry=POLYGON ((0.0000 45.0000, 1.0000 43.0000, 2.0000 43.0000, 3.0000 46.0000, 0.0000 45.0000))&productType=S2MSI1C&maxRecords=20&page=1
2021-01-20 09:59:42,673-15s eodag.core                       [INFO    ] Found 73 result(s) on provider 'peps'


In [10]:
print(
    f"The search has found an estimated number of {estimated_total_nbr_of_results} products matching "
    f"our criteria. It has returned the first {len(products)} ones available."
)

The search has found an estimated number of 73 products matching our criteria. It has returned the first 20 ones available.


The products returned by the search are displayed on the **map above**.The returned products are indeed intersecting with our area of interest.

In [11]:
products_layer = ipyleaflet.GeoJSON(data=products.as_geojson_object(), style=dict(color='red'))
m.add_layer(products_layer)

`eodag` also accepts a WKT search geometry. We use here the WKT representation of the `Shapely` polygon previously created.

In [12]:
aio_wkt = aio_shapely_polyg.wkt
aio_wkt

'POLYGON ((0 45, 1 43, 2 43, 3 46, 0 45))'

In [13]:
products, estimated_total_nbr_of_results = dag.search(
    productType=product_type,
    start=start_date,
    end=end_date,
    geom=aio_wkt
)

2021-01-20 09:59:42,827-15s eodag.core                       [INFO    ] Searching product type 'S2_MSI_L1C' on provider: peps
2021-01-20 09:59:42,828-15s eodag.plugins.search.qssearch    [INFO    ] Sending count request: https://peps.cnes.fr/resto/api/collections/S2ST/search.json?startDate=2018-06-01&completionDate=2018-06-15&geometry=POLYGON ((0.0000 45.0000, 1.0000 43.0000, 2.0000 43.0000, 3.0000 46.0000, 0.0000 45.0000))&productType=S2MSI1C&maxRecords=1&page=1
2021-01-20 09:59:44,491-15s eodag.plugins.search.qssearch    [INFO    ] Sending search request: https://peps.cnes.fr/resto/api/collections/S2ST/search.json?startDate=2018-06-01&completionDate=2018-06-15&geometry=POLYGON ((0.0000 45.0000, 1.0000 43.0000, 2.0000 43.0000, 3.0000 46.0000, 0.0000 45.0000))&productType=S2MSI1C&maxRecords=20&page=1
2021-01-20 09:59:47,615-15s eodag.core                       [INFO    ] Found 73 result(s) on provider 'peps'


In [14]:
print(
    f"The search has found an estimated number of {estimated_total_nbr_of_results} products matching "
    f"our criteria. It has returned the first {len(products)} ones available."
)

The search has found an estimated number of 73 products matching our criteria. It has returned the first 20 ones available.


The products returned by the search are displayed in blue on the **map above**. As expected they are the same as the products returned by the previous search made with the `Shapely` object.

In [15]:
products_layer = ipyleaflet.GeoJSON(data=products.as_geojson_object(),color='blue')
m.add_layer(products_layer)

Note that `eodag` accepts any type of `Shapely` or *WKT* geometry. Our search geometry could thus be a point, a line, etc. The products returned are those which intersect with the provided geometry. There is however a single special case, whenever the input is a `Shapely` multipolygon only the first polygon of the collection is used.

#### Bounding box

In this case the area of interest is defined by its bounding box, i.e. its rectangular extent. This bounding box can be defined as a dictionnary:

In [16]:
aio_bbox = {
    'lonmin': -1.999512,
    'lonmax': 4.570313,
    'latmin': 42.763146,
    'latmax': 46.754917
}

We display the bounding box on a map.

In [17]:
m = ipyleaflet.Map(center=[43.6, 1.5], zoom=5)
search_layer = ipyleaflet.Rectangle(
    bounds=((aio_bbox["latmin"], aio_bbox["lonmin"]), (aio_bbox["latmax"], aio_bbox["lonmax"])),
    color="green"
)
m.add_layer(search_layer)
m

Map(center=[43.6, 1.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

In [18]:
products, estimated_total_nbr_of_results = dag.search(
    productType=product_type,
    start=start_date,
    end=end_date,
    geom=aio_bbox
)

2021-01-20 09:59:47,726-15s eodag.core                       [INFO    ] Searching product type 'S2_MSI_L1C' on provider: peps
2021-01-20 09:59:47,728-15s eodag.plugins.search.qssearch    [INFO    ] Sending count request: https://peps.cnes.fr/resto/api/collections/S2ST/search.json?startDate=2018-06-01&completionDate=2018-06-15&geometry=POLYGON ((-1.9995 42.7631, -1.9995 46.7549, 4.5703 46.7549, 4.5703 42.7631, -1.9995 42.7631))&productType=S2MSI1C&maxRecords=1&page=1
2021-01-20 09:59:49,999-15s eodag.plugins.search.qssearch    [INFO    ] Sending search request: https://peps.cnes.fr/resto/api/collections/S2ST/search.json?startDate=2018-06-01&completionDate=2018-06-15&geometry=POLYGON ((-1.9995 42.7631, -1.9995 46.7549, 4.5703 46.7549, 4.5703 42.7631, -1.9995 42.7631))&productType=S2MSI1C&maxRecords=20&page=1
2021-01-20 09:59:52,984-15s eodag.core                       [INFO    ] Found 190 result(s) on provider 'peps'


In [19]:
print(
    f"The search has found an estimated number of {estimated_total_nbr_of_results} products matching "
    f"our criteria. It has returned the first {len(products)} ones available."
)

The search has found an estimated number of 190 products matching our criteria. It has returned the first 20 ones available.


The products returned by the search are displayed on the **map above**. The returned products are indeed intersecting with our area of interest.

In [20]:
products_layer = ipyleaflet.GeoJSON(data=products.as_geojson_object(), style=dict(color='red'))
m.add_layer(products_layer)

The bounding box can also be defined as a list of coordinates: `[lonmin, latmin, lonmax, latmax]`.

In [21]:
aio_bbox = [-1.999512, 42.763146, 4.570313, 46.754917]

In [22]:
products, estimated_total_nbr_of_results = dag.search(
    productType=product_type,
    start=start_date,
    end=end_date,
    geom=aio_bbox
)

2021-01-20 09:59:53,081-15s eodag.core                       [INFO    ] Searching product type 'S2_MSI_L1C' on provider: peps
2021-01-20 09:59:53,083-15s eodag.plugins.search.qssearch    [INFO    ] Sending count request: https://peps.cnes.fr/resto/api/collections/S2ST/search.json?startDate=2018-06-01&completionDate=2018-06-15&geometry=POLYGON ((-1.9995 42.7631, -1.9995 46.7549, 4.5703 46.7549, 4.5703 42.7631, -1.9995 42.7631))&productType=S2MSI1C&maxRecords=1&page=1
2021-01-20 09:59:54,842-15s eodag.plugins.search.qssearch    [INFO    ] Sending search request: https://peps.cnes.fr/resto/api/collections/S2ST/search.json?startDate=2018-06-01&completionDate=2018-06-15&geometry=POLYGON ((-1.9995 42.7631, -1.9995 46.7549, 4.5703 46.7549, 4.5703 42.7631, -1.9995 42.7631))&productType=S2MSI1C&maxRecords=20&page=1
2021-01-20 09:59:57,803-15s eodag.core                       [INFO    ] Found 190 result(s) on provider 'peps'


In [23]:
print(
    f"The search has found an estimated number of {estimated_total_nbr_of_results} products matching "
    f"our criteria. It has returned the first {len(products)} ones available."
)

The search has found an estimated number of 190 products matching our criteria. It has returned the first 20 ones available.


The products returned by the search are displayed in blue on the **map above**. As expected they are the same as the products returned by the previous search made with the bounding box expressed as a dictionnary.

In [24]:
products_layer = ipyleaflet.GeoJSON(data=products.as_geojson_object(), style=dict(color='blue'))
m.add_layer(products_layer)

### Location search

When we instantiated the `EODataAcessGateway` the logs showed that a *locations configuration* was automatically loaded by `eodag` from its local config directory. A *locations configuration* is a YAML file that contains a shapefile list associated to a name and an attribute. A minimal example of such a file is provided below:

```YAML
shapefiles:

  - name: continent
    path: /path/to/continents.shp
    attr: fullname
```

Where:
* `name` is the argument name that we'll use in a `search` to refer to this specific location.
* `path` is the absolute path to the shapefile
* `attr` is the field of the shapefile we'll use to select features from it

In this minimal example we assume that we have a *continents.shp* shapefile that contains continent's areas (polygons) and a field *fullname* (it may have other fields, they just won't be of any use here). The following search uses the geometry of the feature of *continents.shp* that has *fullname* equal to *Europe*:

```python
products, estimated_total_nbr_of_results = dag.search(productType='S2_MSI_L1C', continent='Europe')
```

The location query (here we searched for `"Europe"`) accepts [regular expressions](https://en.wikipedia.org/wiki/Regular_expression) which can come in handy when the query field has an underlying structure (e.g. tiles).

The locations configuration is stored in the `locations_config` attribute of the `EODataAcessGateway` once instantiated.

In [25]:
dag.locations_config

[{'name': 'country',
  'path': '/home/maxime/.config/eodag/shp/ne_110m_admin_0_map_units.shp',
  'attr': 'ADM0_A3_US'}]

The default configuration contains a [Natural Earth Countries](https://www.naturalearthdata.com/downloads/110m-cultural-vectors/110m-admin-0-countries/) shapefile.

In [26]:
countries_shpfile = dag.locations_config[0]["path"]
attr = dag.locations_config[0]["attr"]

This shapefile has an `ADM0_A3_US` field which describes each country by a short code such as *FRA* for *France* or *JPN* for *Japan*.

In [27]:
import shapefile
with shapefile.Reader(countries_shpfile) as shp:
    print(shp, "\n")
    print(attr, ":\n", *set(rec[attr] for rec in shp.records()))

shapefile Reader
    183 shapes (type 'POLYGON')
    183 records (95 fields) 

ADM0_A3_US :
 DOM MLI MMR KWT KHM BDI BWA BGR GRC KOS GBR IRL COG SRB CIV AFG LKA LAO SLV ERI MWI LSO DZA HTI UKR KEN RUS MDA EGY HND PAN COD BTN BLR MKD BIH BHS USA ECU SVN JAM CHN UGA TUR ISR TLS ATF BEL THA ARE PRI GMB MYS AGO KAZ MRT LBR TUN NAM ALB SUR ITA CAN LVA CUB IRQ KOR NLD SDS NOR GTM POL GAB FJI ESP ZMB ISL GNB MEX CMR SOM CYP KGZ PNG TZA LBN NGA MNE NZL SAH GRL EST ATA SYR CAF MAR URY ZAF LBY TCD PER GEO SAU TWN FRA BOL BEN ARM AUS DJI SDN TGO CRI SVK COL PRY VNM IDN AZE PSX VUT GNQ RWA JOR HUN HRV GIN BGD PHL IND UZB FLK CZE TKM PAK YEM CHE BLZ SEN JPN DNK MNG NPL FIN SLB TTO NIC BRA MDG DEU ZWE IRN GUY PRT NCL BFA GHA ROU ETH LTU NER ARG QAT SWZ LUX BRN PRK TJK SWE OMN MOZ CHL VEN SLE AUT


The *countries* shapefile is displayed on an interactive map, hover over the countries to display their *ISO_A2* code.

In [28]:
m = ipyleaflet.Map(center=(47, 9), zoom=5)
label = ipywidgets.Label(layout=ipywidgets.Layout(width='100%'))

with shapefile.Reader(countries_shpfile) as shp:
    shaperecs = shp.shapeRecords()
    
countries_layer = ipyleaflet.GeoJSON(
    data=shaperecs.__geo_interface__,  # returns a dict representing a GeoJSON object
    hover_style={'color': 'white', 'fillOpacity': 0.5}
)

def hover_handler(event=None, feature=None, id=None, properties=None):
    label.value = properties[attr]

countries_layer.on_hover(hover_handler)
m.add_layer(countries_layer)
ipywidgets.VBox([m, label])

We can define our search area as Switzerland with the value `CHE`:

In [29]:
products, estimated_total_nbr_of_results = dag.search(
    productType=product_type,
    start=start_date,
    end=end_date,
    country='CHE'
)

2021-01-20 09:59:58,187-15s eodag.core                       [INFO    ] Searching product type 'S2_MSI_L1C' on provider: peps
2021-01-20 09:59:58,188-15s eodag.plugins.search.qssearch    [INFO    ] Sending count request: https://peps.cnes.fr/resto/api/collections/S2ST/search.json?startDate=2018-06-01&completionDate=2018-06-15&geometry=POLYGON ((9.5942 47.5251, 9.6329 47.3476, 9.4800 47.1028, 9.9324 46.9207, 10.4427 46.8935, 10.3634 46.4836, 9.9228 46.3149, 9.1829 46.4402, 8.9663 46.0369, 8.4900 46.0052, 8.3166 46.1636, 7.7560 45.8245, 7.2739 45.7769, 6.8436 45.9911, 6.5001 46.4297, 6.0226 46.2730, 6.0374 46.7258, 6.7687 47.2877, 6.7366 47.5418, 7.1922 47.4498, 7.4668 47.6206, 8.3173 47.6136, 8.5226 47.8308, 9.5942 47.5251))&productType=S2MSI1C&maxRecords=1&page=1
2021-01-20 09:59:59,844-15s eodag.plugins.search.qssearch    [INFO    ] Sending search request: https://peps.cnes.fr/resto/api/collections/S2ST/search.json?startDate=2018-06-01&completionDate=2018-06-15&geometry=POLYGON ((9.59

In [30]:
print(
    f"The search has found an estimated number of {estimated_total_nbr_of_results} products matching "
    f"our criteria. It has returned the first {len(products)} ones available."
)

The search has found an estimated number of 89 products matching our criteria. It has returned the first 20 ones available.


The products returned by the search are displayed in red on the **map above**. The returned products are indeed intersecting with Switzerland.

In [31]:
products_layer = ipyleaflet.GeoJSON(
    data=products.as_geojson_object(),
    style={'color': 'red'},
    hover_style={'fillColor': 'pink'}
)
m.add_layer(products_layer)

#### Combined geometry and location search

`eodag` allows to combine a geometry search (e.g. a bounding box) with a location search. In that case the search area becomes the union of the two. In the next example we'll search for products in the bounding box previously defined and in Switzerland. 

In [32]:
m = ipyleaflet.Map(center=(47, 9), zoom=5)
m.add_layer(countries_layer)
m.add_layer(search_layer)
m

Map(center=[47, 9], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [33]:
products, estimated_total_nbr_of_results = dag.search(
    productType=product_type,
    start=start_date,
    end=end_date,
    geom=aio_bbox,
    country='CHE'
)

2021-01-20 10:00:03,595-15s eodag.core                       [INFO    ] Searching product type 'S2_MSI_L1C' on provider: peps
2021-01-20 10:00:03,596-15s eodag.plugins.search.qssearch    [INFO    ] Sending count request: https://peps.cnes.fr/resto/api/collections/S2ST/search.json?startDate=2018-06-01&completionDate=2018-06-15&geometry=MULTIPOLYGON (((9.5942 47.5251, 9.6329 47.3476, 9.4800 47.1028, 9.9324 46.9207, 10.4427 46.8935, 10.3634 46.4836, 9.9228 46.3149, 9.1829 46.4402, 8.9663 46.0369, 8.4900 46.0052, 8.3166 46.1636, 7.7560 45.8245, 7.2739 45.7769, 6.8436 45.9911, 6.5001 46.4297, 6.0226 46.2730, 6.0374 46.7258, 6.7687 47.2877, 6.7366 47.5418, 7.1922 47.4498, 7.4668 47.6206, 8.3173 47.6136, 8.5226 47.8308, 9.5942 47.5251)), ((-1.9995 42.7631, -1.9995 46.7549, 4.5703 46.7549, 4.5703 42.7631, -1.9995 42.7631)))&productType=S2MSI1C&maxRecords=1&page=1
2021-01-20 10:00:05,296-15s eodag.plugins.search.qssearch    [INFO    ] Sending search request: https://peps.cnes.fr/resto/api/colle

In [34]:
print(
    f"The search has found an estimated number of {estimated_total_nbr_of_results} products matching "
    f"our criteria. It has returned the first {len(products)} ones available."
)

The search has found an estimated number of 217 products matching our criteria. It has returned the first 20 ones available.


In [35]:
products_layer = ipyleaflet.GeoJSON(
    data=products.as_geojson_object(),
    style={'color': 'red'},
    hover_style={'fillColor': 'pink'}
)
m.add_layer(products_layer)

The products returned by the search are displayed in red on the **map above**. The returned products are indeed intersecting the bounding box (green) and Switzerland.

#### Custom locations configuration: download products from a Sentinel 2 tile

With respect to the locations configuration we have so far only used the default one provided by `eodag`. It is possible to create your own locations configuration YAML file and use it either by instantiating the `EODataAcessGateway` with `locations_conf_path=path/to/your/location_config.yml` or by setting the `EODAG_LOCS_CFG_FILE` environment variable to the locations configuration file path.

In this example we will use a shapefile that represents the Sentinel 2 tiling grid to search for products at a specific tile. In this shapefile each tile is defined by its centroid and a `tile_id` attribute (e.g. *29PMT*). This shapefile was created by downloading first the Sentinel 2 tiling grid (MGRS) provided [by ESA as a KML file](https://web.archive.org/web/20200907072744/https://sentinel.esa.int/web/sentinel/missions/sentinel-2/data-products). It was then converted as a shapefile and processed to compute the centroids. We use the tile's centroid here as `eodag` returns products that intersects the user defined search area. Since tiles overlap with each other, using the polygons instead of the centroids would return more tiles than just the one we target. 

In [36]:
# We unzip the archived shapefile.
from zipfile import ZipFile

with ZipFile(sentinel2_grid_zip, 'r') as fzip:
    fzip.extractall('auxdata')

We check the content of this shapefile:

In [37]:
sentinel2_shp = os.path.join('auxdata', 'sentinel2_tiling_grid_centroids.shp')
with shapefile.Reader(sentinel2_shp) as shp:
    print(shp, "\n")
    print("fields:", shp.fields)

shapefile Reader
    56686 shapes (type 'POINT')
    56686 records (2 fields) 

fields: [('DeletionFlag', 'C', 1, 0), ['tile_id', 'C', 5, 0]]


It has about 57 000 tiles/polygons and a field `tile_id`.

We create a YAML file to configure this new location selector, we will refer it as `s2_tile_centroid`.

In [38]:
# Save the locations configuration file.
locations_yaml_content = """
shapefiles:
  - name: s2_tile_centroid
    path: {}
    attr: tile_id
""".format(os.path.abspath(sentinel2_shp))

with open(os.path.join(workspace, 'custom_locations.yml'), "w") as f_yml:
    f_yml.write(locations_yaml_content.strip())

In [39]:
dag = EODataAccessGateway(locations_conf_path=os.path.join(workspace, 'custom_locations.yml'))

2021-01-20 10:00:08,661-15s eodag.config                     [INFO    ] Loading user configuration from: /home/maxime/.config/eodag/eodag.yml
2021-01-20 10:00:08,710-15s eodag.core                       [INFO    ] Locations configuration loaded from eodag_workspace_search_area/custom_locations.yml


Our target tile is `31TFK` and is located in the South-East of France. We display its centroid on an interactive map, you can hover over its area to check that it has the right *tile_id*.

In [40]:
m = ipyleaflet.Map(center=(47, 9), zoom=5)

with shapefile.Reader(sentinel2_shp) as shp:
    shaperecs = shp.shapeRecords()

targetted_tile = [
    sr.__geo_interface__
    for sr in shaperecs
    if sr.record["tile_id"] == "31TFK"
]

targetted_tile_layer = ipyleaflet.GeoJSON(
    data=targetted_tile[0],
    hover_style={'color': 'white', 'fillOpacity': 0.5}
)

m.add_layer(targetted_tile_layer)
m

Map(center=[47, 9], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

We search for products that intersect with the centroid of this tile.

In [41]:
products, estimated_total_nbr_of_results = dag.search(
    productType=product_type,
    start=start_date,
    end=end_date,
    s2_tile_centroid="31TFK"
)

2021-01-20 10:00:11,152-15s eodag.core                       [INFO    ] Searching product type 'S2_MSI_L1C' on provider: peps
2021-01-20 10:00:11,153-15s eodag.plugins.search.qssearch    [INFO    ] Sending count request: https://peps.cnes.fr/resto/api/collections/S2ST/search.json?startDate=2018-06-01&completionDate=2018-06-15&geometry=POINT (4.9533 44.6422)&productType=S2MSI1C&maxRecords=1&page=1
2021-01-20 10:00:12,328-15s eodag.plugins.search.qssearch    [INFO    ] Sending search request: https://peps.cnes.fr/resto/api/collections/S2ST/search.json?startDate=2018-06-01&completionDate=2018-06-15&geometry=POINT (4.9533 44.6422)&productType=S2MSI1C&maxRecords=20&page=1
2021-01-20 10:00:13,958-15s eodag.core                       [INFO    ] Found 6 result(s) on provider 'peps'


In [42]:
print(
    f"The search has found an estimated number of {estimated_total_nbr_of_results} products matching "
    f"our criteria. It has returned the first {len(products)} ones available."
)

The search has found an estimated number of 6 products matching our criteria. It has returned the first 6 ones available.


The products returned by the search are displayed in red on the **map above**.The returned products are indeed intersecting with the tile's centroid.

In [43]:
products_layer = ipyleaflet.GeoJSON(
    data=products.as_geojson_object(),
    style={'color': 'red'},
    hover_style={'fillColor': 'pink'}
)

m.add_layer(products_layer)

Note that instead of working with centroids we could have directly used the polygons and filter the returned products to keep only those fully contained within the tile. Check out the advanced tutorial to learn more about crunching products with `eodag`.  

Additionally we can search for products for several tiles using a *regular expression*. We use the expression `"31T[CDE][MLK]"` to look for products over 9 different tiles (*31TCM*, *31TCL*, *31TCK*, *31TDM*, etc.) over France.

In [44]:
products, estimated_total_nbr_of_results = dag.search(
    productType=product_type,
    start=start_date,
    end=end_date,
    s2_tile_centroid="31T[CDE][MLK]"
)

2021-01-20 10:00:15,026-15s eodag.core                       [INFO    ] Searching product type 'S2_MSI_L1C' on provider: peps
2021-01-20 10:00:15,027-15s eodag.plugins.search.qssearch    [INFO    ] Sending count request: https://peps.cnes.fr/resto/api/collections/S2ST/search.json?startDate=2018-06-01&completionDate=2018-06-15&geometry=MULTIPOINT (3.7147 46.4567, 3.7032 45.5565, 3.6922 44.6568, 2.4122 46.4574, 1.1109 46.4433, 1.1413 45.5436, 1.1703 44.6442, 2.4216 45.5572, 2.4306 44.6575)&productType=S2MSI1C&maxRecords=1&page=1
2021-01-20 10:00:16,333-15s eodag.plugins.search.qssearch    [INFO    ] Sending search request: https://peps.cnes.fr/resto/api/collections/S2ST/search.json?startDate=2018-06-01&completionDate=2018-06-15&geometry=MULTIPOINT (3.7147 46.4567, 3.7032 45.5565, 3.6922 44.6568, 2.4122 46.4574, 1.1109 46.4433, 1.1413 45.5436, 1.1703 44.6442, 2.4216 45.5572, 2.4306 44.6575)&productType=S2MSI1C&maxRecords=20&page=1
2021-01-20 10:00:19,263-15s eodag.core                    

In [45]:
print(
    f"The search has found an estimated number of {estimated_total_nbr_of_results} products matching "
    f"our criteria. It has returned the first {len(products)} ones available."
)

The search has found an estimated number of 32 products matching our criteria. It has returned the first 20 ones available.


The products returned by the search are displayed on the map below. Hover over the tiles to check that their name correctly matches with the ones we were after.

In [46]:
m = ipyleaflet.Map(center=(47, 9), zoom=5)
label = ipywidgets.Label(layout=ipywidgets.Layout(width='100%'))

products_layer = ipyleaflet.GeoJSON(
    data=products.as_geojson_object(),
)

def hover_handler(event=None, feature=None, id=None, properties=None):
    label.value = properties["mgrs"]

products_layer.on_hover(hover_handler)
m.add_layer(products_layer)
ipywidgets.VBox([m, label])